<a href="https://colab.research.google.com/github/Sriramkrishna-Deshpande/Prompt-to-Animation-Manim-Animator-using-LLMs/blob/main/Manim_Generator_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**First run the below cell to install pre requistes before starting.**
*It will consume typically 3-4 mins and then  
*  **Restart the session**
*  **Get Your API Key FROM Here  :[Groq API Keys ](https://console.groq.com/playground?model=llama-3.3-70b-versatile)**

In [ ]:
!sudo apt update
!sudo apt install libcairo2-dev \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython==8.21.0
!pip install langchain-groq  langchain-community


*Import All Nessary Libraries*

In [ ]:
import os
import subprocess
from langchain_core.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from IPython.display import Video, display


In [ ]:
 Set your Groq API key
os.environ["GROQ_API_KEY"] =" "#Enter your API Key

# Initialize the LLM
llm = ChatGroq(
    temperature=0.7,  # lower for better consistency
    model="llama-3-70b-8192-versatile",
    groq_api_key=os.environ["GROQ_API_KEY"]
)

# Define a stricter, smarter prompt
prompt = PromptTemplate(
    input_variables=["user_input"],
    template="""
You are a Manim CE v0.19.0 animation expert. Given a description, generate clean, executable Python code with:
- Only this import: from manim import *
- Use: import math, import numpy as np if needed
- One class only: GeneratedScene(Scene)
- No comments, markdown, text outside the code
- No placeholders or explanations

The code must run directly with: manim -ql GeneratedScene.py GeneratedScene

User description:
{user_input}

Now output only valid Manim Python code below:
"""
)

# Create the main chain
chain = (
    {"user_input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

def inject_missing_imports(code):
    """
    Ensure essential imports like `math` and `numpy` are present.
    """
    required = []
    if "math." in code and "import math" not in code:
        required.append("import math")
    if ("np." in code or "numpy" in code) and "import numpy as np" not in code:
        required.append("import numpy as np")
    return "\n".join(required) + "\n" + code if required else code

def run_manim_code(manim_code, output_dir="manim_output", scene_name="GeneratedScene"):
    try:
        if "from manim import *" not in manim_code or f"class {scene_name}(Scene):" not in manim_code:
            return "Error: Generated code is invalid (missing imports or scene class)"

        os.makedirs(output_dir, exist_ok=True)

        # Save code
        code_file = os.path.join(output_dir, f"{scene_name}.py")
        with open(code_file, "w") as f:
            f.write(manim_code)

        output_video_dir = os.path.join(output_dir, "videos", scene_name, "480p15")
        manim_command = [
            "manim", "-ql", "--media_dir", output_dir,
            code_file, scene_name
        ]

        result = subprocess.run(
            manim_command,
            capture_output=True,
            text=True
        )

        if result.returncode != 0:
            return f"RuntimeError: {result.stderr.strip()}"

        video_file = os.path.join(output_video_dir, f"{scene_name}.mp4")
        return video_file if os.path.exists(video_file) else "Error: Video not found."

    except Exception as e:
        return f"Exception: {str(e)}"

def auto_repair_code(broken_code, error_msg):
    """
    Use the LLM to revise based on the error.
    """
    fix_prompt = f"""
The following Manim code failed with the given error. Fix it.

Code:
{broken_code}

Error:
{error_msg}

Generate corrected Manim CE v0.19.0 code, no comments or explanations.
"""
    return llm.invoke(fix_prompt)

def main():
    user_input = input("Enter the description of the animation: ")

    for attempt in range(3):
        print(f"\nAttempt {attempt + 1}...")

        try:
            manim_code = chain.invoke(user_input)
            manim_code = inject_missing_imports(manim_code)

            result = run_manim_code(manim_code)

            if result.endswith(".mp4"):
                print("Animation Rendered:", result)
                display(Video(result, embed=True))
                return
            else:
                print(" Error:", result)
                manim_code = auto_repair_code(manim_code, result)

        except Exception as e:
            print(f"Exception during generation: {e}")

    print("\nAll attempts failed. Please refine your prompt or check the code manually.")

if __name__ == "__main__":
    main()
